In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/data-dictionary', './input/Country-data']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# ALEX: remove path printing
# import os
# for dirname, _, filenames in os.walk('./input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Load libraries and read the data

In [2]:
# ALEX: remove plotting
# import pandas as pd
# import numpy as np

# import seaborn as sns
# import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

# For scaling the data
# ALEX: remove ML code
# from sklearn.preprocessing import StandardScaler

# # To perform K-means clustering
# from sklearn.cluster import KMeans


# #To perform hierarchical clustering
# from scipy.cluster.hierarchy import linkage
# from scipy.cluster.hierarchy import dendrogram
# from scipy.cluster.hierarchy import cut_tree

In [3]:
data_dict = pd.read_csv('./input/data-dictionary.scaled.csv')

In [4]:
data_dict

,Column Name,Description
0,country,Name of the country
1,child_mort,Death of children under 5 years of age per 1000 live births
2,exports,Exports of goods and services per capita. Given as %age of the GDP per capita
3,health,Total health spending per capita. Given as %age of GDP per capita
4,imports,Imports of goods and services per capita. Given as %age of the GDP per capita
5,Income,Net income per person
6,Inflation,The measurement of the annual growth rate of the Total GDP
7,life_expec,The average number of years a new born child would live if the current mortality patterns are to remain the same
8,total_fer,The number of children that would be born to each woman if the current age-fertility rates remain the same.
9,gdpp,The GDP per capita. Calculated as the Total GDP divided by the total population.


In [5]:
countries = pd.read_csv('./input/Country-data.scaled.csv')

In [6]:
countries.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


In [7]:
countries.shape

(167, 10)

Since the exports, healths and imports are given as percentage of GDPP. First we will convert them to their actual values respectively. This will help us to get the actual comparisons among the countries.


In [8]:
# features = ['exports', 'health', 'imports']
# for feature in features:
#     countries[feature] = countries[feature]*countries['gdpp']/100
countries[['exports', 'health', 'imports']]= \
                                countries[['exports', 'health', 'imports']].multiply(countries["gdpp"]/100, axis="index")


In [9]:
countries.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553
1,Albania,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460
3,Angola,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200


In [10]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     167 non-null    object 
 1   child_mort  167 non-null    float64
 2   exports     167 non-null    float64
 3   health      167 non-null    float64
 4   imports     167 non-null    float64
 5   income      167 non-null    int64  
 6   inflation   167 non-null    float64
 7   life_expec  167 non-null    float64
 8   total_fer   167 non-null    float64
 9   gdpp        167 non-null    int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 13.2+ KB


In [11]:
#checking for duplicates
countries.duplicated(subset = ['country'], keep = False).sum()

0

There are no duplicated rows

In [12]:
# Checking for null values in the column
countries.isnull().sum()

country       0
child_mort    0
exports       0
health        0
imports       0
income        0
inflation     0
life_expec    0
total_fer     0
gdpp          0
dtype: int64

There are no NULL values in the dataset

#### Checking for outliers and treating it

In [13]:
countries.describe(percentiles=[.25,.5,.75,.90,.95,.99])

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
count,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,38.270060,7420.618847,1056.733204,6588.352108,17144.688623,7.781832,70.555689,2.947964,12964.155689
std,40.328931,17973.885795,1801.408906,14710.810418,19278.067698,10.570704,8.893172,1.513848,18328.704809
min,2.600000,1.076920,12.821200,0.651092,609.000000,-4.210000,32.100000,1.150000,231.000000
25%,8.250000,447.140000,78.535500,640.215000,3355.000000,1.810000,65.300000,1.795000,1330.000000
50%,19.300000,1777.440000,321.886000,2045.580000,9960.000000,5.390000,73.100000,2.410000,4660.000000
75%,62.100000,7278.000000,976.940000,7719.600000,22800.000000,10.750000,76.800000,3.880000,14050.000000
90%,100.220000,17760.600000,3825.416000,15034.280000,41220.000000,16.640000,80.400000,5.322000,41840.000000
95%,116.000000,31385.100000,4966.701000,24241.560000,48290.000000,20.870000,81.400000,5.861000,48610.000000
99%,153.400000,64794.260000,8410.330400,55371.390000,84374.000000,41.478000,82.370000,6.563600,79088.000000


### Performing Univariate Ananlysis on all the features

In [14]:
features = countries.columns
features

Index(['country', 'child_mort', 'exports', 'health', 'imports', 'income',
       'inflation', 'life_expec', 'total_fer', 'gdpp'],
      dtype='object')

In [15]:
# ALEX: remove plotting
# fig = plt.figure(figsize=(15,25))
for plot, feature in enumerate(features[1:]):
# ALEX: remove plotting
#     fig.add_subplot(5,3,plot+1)
#     sns.boxplot(x=countries[feature], data=countries)
    _ = countries[feature]
                 

From the above plots, we can conculde the following:
- There are outliers in each features.
- Most of the outliers are in health, exports and gdpp
- There are only 167 rows(countries), so removing these outliers will delete the crucial information from the datesets.
- Specially the countries which have exceptionally high child mortalities(outliers), will be missed, which actually need the aid from the NGO. For example, in case of Child Mortality rate, country with value `208` is an outlierm, however, the coutry itself could in dire need of aid.
- So I chose to treat the outliers only in the `gdpp` column. The coutries with high `gdpp` would not require an aid. So I will remove those from the dataset.

In [16]:
#Caluclating the quantile1 and quantile3 for gdpp and removing the upper outliers
Q1 = countries.gdpp.quantile(0.05)
Q3 = countries.gdpp.quantile(0.95)

countries[(countries.gdpp > Q3)]

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
7,Australia,4.8,10276.2,4530.87,10847.1,41400,1.160,82.0,1.93,51900
44,Denmark,4.1,29290.0,6612.00,25288.0,44000,3.220,79.5,1.87,58000
73,Ireland,4.2,50161.0,4475.53,42125.5,45700,-3.220,80.4,2.05,48700
91,Luxembourg,2.8,183750.0,8158.50,149100.0,91700,3.620,81.3,1.63,105000
110,Netherlands,4.5,36216.0,5985.70,31990.8,45500,0.848,80.7,1.79,50300
114,Norway,3.2,34856.6,8323.44,25023.0,62300,5.950,81.0,1.95,87800
123,Qatar,9.0,43796.9,1272.43,16731.4,125000,6.980,79.5,2.07,70300
144,Sweden,3.0,24070.2,5017.23,21204.7,42900,0.991,81.5,1.98,52100
145,Switzerland,4.5,47744.0,8579.00,39761.8,55500,0.317,82.2,1.52,74600


###### In the above table we can see, that Qatar is having a high child mortality rate of 9.0, but its other features like gdpp, total_fertility rate, income etc are in the higher range. So it would not be first in the list of needy countries.

After removing the outlier

In [17]:
countries = countries[(countries.gdpp <= Q3)]

In [18]:
countries.shape

(158, 10)

In [19]:
countries.head(5)

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553
1,Albania,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460
3,Angola,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200


#### Visulaizing the data using Pair Plot

In [20]:
# ALEX: remove plotting
# fig = plt.figure(figsize=(20,35))
# sns.pairplot(countries,diag_kind='kde',corner=True)

From the above plots we can conclude the following about gddp
- It is highly and positively correlated with life expectancy and income
- It is inversely correlated with child_mortality, total fertility and inflation
- health, imports and exports are also positively correlated with gddp
- Also there is linear realtionship between gdpp vs income,imports vs exports, total_fer  vs child_mort

### Top 10 under developed coutries based on the data visulaization 

###### Top 10 coutries with highest mortality rate

In [21]:
# ALEX: remove plotting
# plt.figure(figsize = (15,5))
child_mort_top10 = countries[['country','child_mort']].sort_values('child_mort', ascending = False).head(10)
# ALEX: remove plotting
# ax=sns.barplot(x='country', y='child_mort', data= child_mort_top10)

# for p in ax.patches:
#     ax.annotate(str(p.get_height()), (p.get_x() * 1.05 , p.get_height() * 1.01))
# ax.set(xlabel = '', ylabel= 'Child Mortality Rate')
# plt.xticks(rotation=45)

###### Top 10 countries with the `lowest` spending on health

In [22]:
# ALEX: remove plotting
# plt.figure(figsize = (15,5))
health = countries[['country','health']].sort_values('health', ascending = True).head(10)
# ALEX: remove plotting
# ax = sns.barplot(x='country', y='health', data= health)
# ax.set(xlabel = '', ylabel= 'Health')
# plt.xticks(rotation=90)

###### Top 10 countries with lowest per capita income.

In [23]:
# ALEX: remove plotting
# plt.figure(figsize = (15,5))
income_lowets_10 = countries[['country','income']].sort_values('income', ascending = True).head(10)
# ALEX: remove plotting
# axes = sns.barplot(x='country', y='income', data= income_lowets_10)
# axes.set(xlabel = '', ylabel= 'Per Capita Income')
# plt.xticks(rotation=90)

###### Top 10 Countries with the highest Inflation rate.

In [24]:
# ALEX: remove plotting
# plt.figure(figsize = (15,5))
high_inflation = countries[['country','inflation']].sort_values('inflation', ascending = False).head(10)
# ALEX: remove plotting
# ax = sns.barplot(x='country', y='inflation', data= high_inflation)
# ax.set(xlabel = '', ylabel= 'Inflation')
# plt.xticks(rotation=90)


From the above plot we can see that is Nigeria is clearly standing out having the highest infaltion.

#### From the above visulizations we get a good idea about the under developed countries which might be in need of the NGOs aid.


#### Checking if the datset is fit for applying clustering on it

In [25]:
#Find the Hopkins statistic score
# ALEX: remove ML code
# from sklearn.neighbors import NearestNeighbors
# from random import sample
# from numpy.random import uniform
# import numpy as np
# from math import isnan
 
def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
# ALEX: remove ML code
#     m = int(0.1 * n) 
#     nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
#     rand_X = sample(range(0, n, 1), m)
 
#     ujd = []
#     wjd = []
#     for j in range(0, m):
#         u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
#         ujd.append(u_dist[0][1])
#         w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
#         wjd.append(w_dist[0][1])
 
#     H = sum(ujd) / (sum(ujd) + sum(wjd))
#     if isnan(H):
#         print(ujd, wjd)
#         H = 0
 
#     return H
    _ = X.values

In [26]:
hopkins(countries.drop('country', axis = 1))

### We have got the Hopkins score close to 1 which tends to indicate the data is highly clustered

### Preparing the dataset for clustering

#### Scaling the dataset

In [27]:
features = countries.columns
# ALEX: remove ML code
# standard_scaler = StandardScaler()
# countries_scaled= standard_scaler.fit_transform(countries.drop('country', axis=1))
countries_scaled= countries.drop('country', axis=1)
countries_scaled

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553
1,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090
2,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460
3,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530
4,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200
...,...,...,...,...,...,...,...,...,...
162,29.2,1384.02,155.9250,1565.190,2950,2.62,63.0,3.50,2970
163,17.1,3847.50,662.8500,2376.000,16500,45.90,75.4,2.47,13500
164,23.3,943.20,89.6040,1050.620,4490,12.10,73.1,1.95,1310
165,56.3,393.00,67.8580,450.640,4480,23.60,67.5,4.67,1310


In [28]:
country_norm = pd.DataFrame(countries_scaled)
country_norm.columns = features[1:]
country_norm.head()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553
1,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090
2,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460
3,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530
4,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200


#### Finding the  Silhouette score

In [29]:
# ALEX: remove ML code, plotting
# from sklearn.metrics import silhouette_score
# ss = []
# for k in range(2, 15):
#     kmean = KMeans(n_clusters = k).fit(country_norm)
#     ss.append([k, silhouette_score(country_norm, kmean.labels_)])
# temp = pd.DataFrame(ss)    
# plt.plot(temp[0], temp[1])

#### Elbow score

In [30]:
# sum of squared distances
# ALEX: remove ML code, plotting
# ssd = []
# for num_clusters in list(range(1,21)):
#     model_clus = KMeans(n_clusters = num_clusters, max_iter=50)
#     model_clus.fit(country_norm)
#     ssd.append(model_clus.inertia_)

    
# fig, ax = plt.subplots()
# ax.plot(range(1,21),ssd, marker = '+',color = 'red' , ls = '--', markeredgecolor = 'blue', markersize = '10')

###### By looking silhouette plot, we see the highest peak is at k = 4 and in sum of squared distances plot , we see that the elbow is in the range of 3 to 5 , so let us take k as 4.

### Performing K-Means Clustering on the scaled dataset


In [31]:
# ALEX: remove ML code
# kmeans_clus4 = KMeans(n_clusters=4, max_iter=50, random_state= 50)
# kmeans_clus4.fit(country_norm)

In [32]:
# ALEX: remove ML code
# kmeans_clus4.labels_

In [33]:
#Finding the labels and converting it to series so that it can be appended to the `original countries` easily.
# ALEX: remove ML code
# labels_kmeans = pd.Series(kmeans_clus4.labels_, index=countries.index)

0      0
1      0
2      0
3      0
4      2
      ..
162    0
163    2
164    0
165    0
166    0
Length: 158, dtype: int64

In [34]:
#Adding the labels to countries dataset
countries_kmeans = countries.copy()
# ALEX: remove ML code
# countries_kmeans['Labels'] = labels_kmeans
countries_kmeans.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp,Labels
0,Afghanistan,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553,0
1,Albania,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090,0
2,Algeria,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460,0
3,Angola,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530,0
4,Antigua and Barbuda,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200,2


### Checking the number of countries assigned to each clusters

In [35]:
# ALEX: remove ML code
# countries_kmeans['Labels'].value_counts()

Labels
0    96
2    38
1    23
3     1
Name: count, dtype: int64

## Profiling the clusters based on  - [gdpp, child_mort and income] 

#### Bivariate analysis of clustered countries 

In [36]:
# ALEX: remove plotting
# axes = sns.scatterplot(x='gdpp',y='child_mort',hue='Labels',legend='full',data=countries_kmeans, palette=['green','orange','brown','red'])
# axes.set(xlabel = 'GDPP', ylabel= 'Child Mortality Rate')

In [37]:
# ALEX: remove plotting
# axes = sns.scatterplot(x='gdpp',y='income',hue='Labels',legend='full',data=countries_kmeans, palette=['green','orange','brown','red'])
# axes.set(xlabel = 'GDPP', ylabel= 'Income')

In [38]:
# ALEX: remove plotting
# axes= sns.scatterplot(x='income',y='child_mort',hue='Labels',legend='full',data=countries_kmeans, palette=['green','orange','brown','red'])
# axes.set(xlabel = 'Child Mortality Rate', ylabel= 'Income')

### From the above 3 scatterplots we can deduce the following insights about the clusters, specially about the clster labeled 1.

- In the gdpp vs child_mort plot, the child_mort is high when gdpp is low.
- In the gdpp vs income, when the gdpp is average the income is average.
- In the income vs child_mort, when income is high the child mortality is low and vice versa.

#### Taking the means and analyzing the clusters further

In [39]:
# ALEX: remove ML code
# child_mort_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).child_mort.mean())
# exports_mean=pd.DataFrame(countries_kmeans.groupby(["Labels"]).exports.mean())
# health_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).health.mean())
# imports_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).imports.mean())
# income_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).income.mean())
# inflat_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).inflation.mean())
# life_expec_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).life_expec.mean())
# total_fer_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).total_fer.mean())
# gdpp_mean =pd.DataFrame(countries_kmeans.groupby(["Labels"]).gdpp.mean())

In [40]:
# ALEX: remove ML code
# cluster_analysis_df = pd.concat([pd.Series([0,1,2,3]),child_mort_mean,exports_mean,health_mean,imports_mean,income_mean,inflat_mean,life_expec_mean,
#                                  total_fer_mean,gdpp_mean], axis=1)

In [41]:
# ALEX: remove ML code
# cluster_analysis_df.columns = ["Labels","child_mort_mean","exports_mean","health_mean","imports_mean","income_mean","inflation_mean","life_expec_mean","total_fer_mean","gdpp_mean"]
# cluster_analysis_df

,Labels,child_mort_mean,exports_mean,health_mean,imports_mean,income_mean,inflation_mean,life_expec_mean,total_fer_mean,gdpp_mean
0,0,58.356250,881.680495,152.453189,1128.790543,5266.281250,9.414542,65.688542,3.703229,2433.687500
1,1,6.182609,16439.365217,3186.460870,14321.452174,42130.434783,4.591913,79.639130,1.912174,35443.478261
2,2,15.889474,5871.992632,888.165237,5772.137105,20065.789474,7.114105,74.584211,1.970526,12391.578947
3,3,2.800000,93200.000000,1845.360000,81084.000000,72100.000000,-0.046000,82.700000,1.150000,46600.000000


In [42]:
# ALEX: remove ML code, plotting
# features = cluster_analysis_df.columns
# fig = plt.figure(figsize=(18,15))
# for plot, feature in enumerate(features[1:]):
#     fig.add_subplot(3,3,plot+1)
#     sns.barplot(cluster_analysis_df.Labels, cluster_analysis_df[feature])

From the above plot we can see that the criterias , namely, `gdpp, income and child mortality` are satified by cluster labeled 1 

In [43]:
#Binning the clusters to get the top 5 countries sorted in order exaplined above
# ALEX: remove ML code
# clustered_kmeans=countries[countries_kmeans['Labels']==1]
# clustered_kmeans.sort_values(['gdpp','income','child_mort','health','inflation','life_expec','total_fer','imports','exports'], 
#                       ascending=[True,True,False,True,False,True,False,False,True]).head(10)


,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
115,Oman,11.7,12680.1,534.61,7951.6,45300,15.600,76.1,2.90,19300
128,Saudi Arabia,15.7,9572.8,827.97,6369.0,45400,17.200,75.1,2.96,19300
11,Bahrain,8.6,14386.5,1028.79,10536.3,41100,7.440,76.0,2.16,20700
98,Malta,6.8,32283.0,1825.15,32494.0,28300,3.830,80.3,1.36,21100
135,Slovenia,3.2,15046.2,2201.94,14718.6,28700,-0.987,79.5,1.57,23400
74,Israel,4.6,10710.0,2334.78,10067.4,29600,1.770,81.4,3.03,30600
139,Spain,3.8,7828.5,2928.78,8227.6,32500,0.160,81.9,1.37,30700
42,Cyprus,3.6,15461.6,1838.76,17710.0,33900,2.010,79.9,1.42,30800
111,New Zealand,6.2,10211.1,3403.70,9436.0,32300,3.730,80.9,2.17,33700
157,United Arab Emirates,8.6,27195.0,1281.00,22260.0,57600,12.500,76.5,1.87,35000


### So from the above results and analysis, we can conclude that as per K- means clustering, the country which are direst need of aid are:
    1. Burundi
    2. Liberia
    3. Congo, Dem. Rep
    4. Niger
    5. Sierra Leone

##  Taking Hierarchical Clustering approach

#### Using the already cleaned and scaled data, done earlier in  kmeans analysis

In [44]:
country_norm.shape

(158, 9)

In [45]:
country_norm.head()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553
1,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090
2,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460
3,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530
4,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200


In [46]:
countries_hrcl  = country_norm.copy()
# ALEX: remove plotting
# fig = plt.figure(figsize = (25,8))
# mergings = linkage(countries_hrcl, method = "single", metric='euclidean')
# dendrogram(mergings)
# plt.show()

### By single method heirarchical clustering things are not clear, so let us go for complete heirarchical clustering method.

In [47]:
# HC with complete method
# ALEX: remove plotting
# fig = plt.figure(figsize = (25, 8))
# mergings = linkage(countries_hrcl, method = "complete", metric='euclidean')
# dendrogram(mergings)
# plt.show()

##### At 12.5 there are 3 clusters and at 10.0 there are 4 clusters
### Let us cut the dendrogram  such that there are  n_clusters = 4 


In [48]:
countries

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553
1,Albania,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460
3,Angola,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200
...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,29.2,1384.02,155.9250,1565.190,2950,2.62,63.0,3.50,2970
163,Venezuela,17.1,3847.50,662.8500,2376.000,16500,45.90,75.4,2.47,13500
164,Vietnam,23.3,943.20,89.6040,1050.620,4490,12.10,73.1,1.95,1310
165,Yemen,56.3,393.00,67.8580,450.640,4480,23.60,67.5,4.67,1310


In [49]:
# ALEX: remove ML code
# clusterCut = cut_tree(mergings, n_clusters = 4).reshape(-1,)
# clusterCut

In [50]:
countries_hrcl_cluster = countries.copy()
countries_hrcl_cluster.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553
1,Albania,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460
3,Angola,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200


In [51]:
# ALEX: remove ML code
# countries_hrcl_cluster['Labels'] = pd.Series(clusterCut, index=countries.index)
countries_hrcl_cluster.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp,Labels
0,Afghanistan,90.2,55.30,41.9174,248.297,1610,9.44,56.2,5.82,553,0
1,Albania,16.6,1145.20,267.8950,1987.740,9930,4.49,76.3,1.65,4090,0
2,Algeria,27.3,1712.64,185.9820,1400.440,12900,16.10,76.5,2.89,4460,0
3,Angola,119.0,2199.19,100.6050,1514.370,5900,22.40,60.1,6.16,3530,0
4,Antigua and Barbuda,10.3,5551.00,735.6600,7185.800,19100,1.44,76.8,2.13,12200,2


### Checking the number of countries assigned to each clusters

In [52]:
# ALEX: remove ML code
# countries_hrcl_cluster['Labels'].value_counts()

Labels
0    96
2    38
1    23
3     1
Name: count, dtype: int64

The above numbers are quite different from the one which we got in kMeans method.
But that is understandable becuase the Hierarchical clustering method will form different clusters than the Kmeans one.

## Profiling the clusters based on  - [gdpp, child_mort and income] 

#### Bivariate analysis of clustered countries 

In [53]:
# ALEX: remove plotting
# axes = sns.scatterplot(x='gdpp', y='child_mort', hue='Labels',legend='full',data=countries_hrcl_cluster, palette=['green','orange','brown','red'])
# axes.set(xlabel = 'GDPP', ylabel= 'Child Mortality Rate')

In [54]:
# ALEX: remove plotting
# axes = sns.scatterplot(x='gdpp',y='income',hue='Labels',legend='full',data=countries_hrcl_cluster, palette=['green','orange','brown','red'])
# axes.set(xlabel = 'GDPP', ylabel= 'Income')

In [55]:
# ALEX: remove plotting
# axes= sns.scatterplot(x='income',y='child_mort',hue='Labels',legend='full',data=countries_hrcl_cluster, palette=['green','orange','brown','red'])
# axes.set(xlabel = 'Child Mortality Rate', ylabel= 'Income')

### From the above 3 scatterplots we can deduce the following insights about the clusters, specially about the clster labeled 0.

- In the gdpp vs child_mort plot, the child_mort is high when gdpp is low.
- In the gdpp vs income, when the gdpp is average the income is average.
- In the income vs child_mort, when income is high the child mortality is low and vice versa.

#### Taking the means of the features and analyzing the clusters further

In [56]:
# ALEX: remove ML code
# child_mort_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).child_mort.mean())
# exports_mean=pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).exports.mean())
# health_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).health.mean())
# imports_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).imports.mean())
# income_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).income.mean())
# inflat_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).inflation.mean())
# life_expec_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).life_expec.mean())
# total_fer_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).total_fer.mean())
# gdpp_mean =pd.DataFrame(countries_hrcl_cluster.groupby(["Labels"]).gdpp.mean())

In [57]:
# ALEX: remove ML code
# hrcl_cluster_analysis_df = pd.concat([pd.Series([0,1,2,3]),child_mort_mean,exports_mean,health_mean,imports_mean,income_mean,inflat_mean,life_expec_mean,
#                                  total_fer_mean,gdpp_mean], axis=1)

In [58]:
# ALEX: remove ML code
# hrcl_cluster_analysis_df.columns = ["Labels","child_mort_mean","exports_mean","health_mean","imports_mean","income_mean","inflation_mean","life_expec_mean","total_fer_mean","gdpp_mean"]
# hrcl_cluster_analysis_df

,Labels,child_mort_mean,exports_mean,health_mean,imports_mean,income_mean,inflation_mean,life_expec_mean,total_fer_mean,gdpp_mean
0,0,58.356250,881.680495,152.453189,1128.790543,5266.281250,9.414542,65.688542,3.703229,2433.687500
1,1,6.182609,16439.365217,3186.460870,14321.452174,42130.434783,4.591913,79.639130,1.912174,35443.478261
2,2,15.889474,5871.992632,888.165237,5772.137105,20065.789474,7.114105,74.584211,1.970526,12391.578947
3,3,2.800000,93200.000000,1845.360000,81084.000000,72100.000000,-0.046000,82.700000,1.150000,46600.000000


In [59]:
# ALEX: remove ML code, plotting
# features = hrcl_cluster_analysis_df.columns
# fig = plt.figure(figsize=(18,15))
# for plot, feature in enumerate(features[1:]):
#     fig.add_subplot(3,3,plot+1)
#     sns.barplot(hrcl_cluster_analysis_df.Labels, hrcl_cluster_analysis_df[feature])

In [60]:
#Binning the clusters to get the top 5 countries sorted in order exaplined above
# ALEX: remove ML code
# clustered_hrcl=countries[countries_hrcl_cluster['Labels']==0]
# clustered_hrcl.sort_values(['gdpp','income','child_mort','health','inflation','life_expec','total_fer','imports','exports'], 
#                       ascending=[True,True,False,True,False,True,False,False,True]).head(10)


,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
26,Burundi,93.6,20.6052,26.7960,90.552,764,12.30,57.7,6.26,231
88,Liberia,89.3,62.4570,38.5860,302.802,700,5.47,60.8,5.02,327
37,"Congo, Dem. Rep.",116.0,137.2740,26.4194,165.664,609,20.80,57.5,6.54,334
112,Niger,123.0,77.2560,17.9568,170.868,814,2.55,58.8,7.49,348
132,Sierra Leone,160.0,67.0320,52.2690,137.655,1220,17.20,55.0,5.20,399
93,Madagascar,62.2,103.2500,15.5701,177.590,1390,8.79,60.8,4.60,413
106,Mozambique,101.0,131.9850,21.8299,193.578,918,7.64,54.5,5.56,419
31,Central African Republic,149.0,52.6280,17.7508,118.190,888,2.01,47.5,5.21,446
94,Malawi,90.5,104.6520,30.2481,160.191,1030,12.10,53.1,5.31,459
50,Eritrea,55.2,23.0878,12.8212,112.306,1420,11.60,61.7,4.61,482


### So from the above results and analysis, we can conclude that as per Heirarchical clustering, the country which are direst need of aid are:
    1. Burundi
    2. Liberia
    3. Congo, Dem. Rep
    4. Niger
    5. Sierra Leone

### We got same countries by both K-means and Heirarchical Clustering techniques: 
##### Therefore following are the countries which are in direst need of aid by considering socio – economic factor into consideration:

    1. Burundi
    2. Liberia
    3. Congo, Dem. Rep.
    4. Niger
    5. Sierra Leone